# import libraries

In [ ]:
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.7/316.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 36.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 21.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394487 sha256=8371cb3498d0898f5117781224615060e92c6b8362e671f394bbac7cc6cb52f6
  Stored in directory: /root/.cache/pip/wheels/ac/62/f6/88933dadcd64a1614894614aa68cf57c4b8e5256acb650b1f1
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp39-cp39-linux_x86_64.whl size=180369 sha256=94895ba11ccc7a0a577027596bcdb1133d021f61d2c942e19b03e25680fe7694
  Stored in directory: /root/.cache/pip/wheels/8e/ff/82/9326b96f96f47472e02c453697b225813e4650c0ed4df2cd49
Successfully built nltk libwapiti
  Attempting unin

In [ ]:
!pip install emojis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install URLExtract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from hazm import *
from urlextract import URLExtract
import emojis

In [1]:
from __future__ import unicode_literals
import tensorflow as tf
from keras.models import Sequential
import pandas as pd
from keras.layers import Dense
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import Adadelta,Adam,RMSprop
from keras.utils import np_utils
from tqdm import tqdm
from nltk.tokenize import word_tokenize


In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# loading data and picking 20% of it

In [ ]:
corpus = pd.read_csv("Snappfood - Sentiment Analysis.csv", on_bad_lines = 'skip', sep='\t')

In [ ]:
corpus

,Unnamed: 0,comment,label,label_id
0,NaN,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1.0
1,NaN,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0.0
2,NaN,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1.0
3,NaN,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0.0
4,NaN,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0.0
...,...,...,...,...
69995,NaN,سلام من به فاکتور غذاهایی که سفارش میدم احتیاج...,SAD,1.0
69996,NaN,سایز پیتزا نسبت به سفارشاتی که قبلا گذشتم کم ش...,SAD,1.0
69997,NaN,من قارچ اضافه رو اضافه کرده بودم بودم اما اگر ...,HAPPY,0.0
69998,NaN,همرو بعد ۲ساعت تاخیر اشتباه آوردن پولشم رفت رو...,SAD,1.0


In [ ]:
df_percent_sad = corpus[corpus["label"] == 'SAD'].sample(n = 3500)
df_percent_happy = corpus[corpus["label"] == 'HAPPY'].sample(n = 3500)

In [ ]:
df_merged = df_percent_happy.append(df_percent_sad, ignore_index=True)

<ipython-input-15-26fbf334edf5>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_merged = df_percent_happy.append(df_percent_sad, ignore_index=True)


In [ ]:
corpus = df_merged.sample(frac=1) 

In [ ]:
corpus

,Unnamed: 0,comment,label,label_id
1252,NaN,غذا سرد شده بود. حیف اون پیتزا که سرد شد,HAPPY,0.0
1949,NaN,مزه خورشت سبزی خوب نبود، غذا مزه‌ی سوختگی می‌د...,HAPPY,0.0
6977,NaN,غذاشون خوب بود ولی پول برنج ایرانی گرفتن برنج ...,SAD,1.0
5275,NaN,بخش پیتزای پنجره‌ای خوش طعم اما بسیار نازک بود...,SAD,1.0
3944,NaN,کیک وانیلی سفارش داده بودم اصلا تازه و خوشمزه ...,SAD,1.0
...,...,...,...,...
1224,NaN,سلام و خسته نباشید مثل همیشه عالی، تازه و خوشم...,HAPPY,0.0
2598,NaN,بسیار زمان تحویل مناسب بود,HAPPY,0.0
923,NaN,مثل همیشه بی نظیر و عالی,HAPPY,0.0
4025,NaN,سالاد سزار بسیار بی کیفیت بود …حیف اسمه به این...,SAD,1.0


# preprocessing

In [ ]:
def _multiple_replace(mapping, text):
    pattern = "|".join(map(re.escape, mapping.keys()))
    return re.sub(pattern, lambda m: mapping[m.group()], str(text))

def convert_fa_numbers(input_str):
    mapping = {
        '۰': '0',
        '۱': '1',
        '۲': '2',
        '۳': '3',
        '۴': '4',
        '۵': '5',
        '۶': '6',
        '۷': '7',
        '۸': '8',
        '۹': '9',
        '.': '.',
    }
    return _multiple_replace(mapping, input_str)

def convert_ar_characters(input_str):
    """
    Converts Arabic chars to related Persian unicode char
    :param input_str: String contains Arabic chars
    :return: New str with converted arabic chars
    """
    mapping = {
        'ك': 'ک',
        'ى': 'ی',
        'ي': 'ی',
        'ئ':'ی',
        'إ':'ا',
        'أ':'ا',
        'ة':'ه',
        'ؤ':'و'
    }
    return _multiple_replace(mapping, input_str)


def preprocess(text):
    extractor = URLExtract()
    for url in extractor.gen_urls(text):
        text = text.replace(url,'<URL>')
    emj = emojis.get(text)
    for i in emj:
        if i in text:
            text = text.replace(i,'<emoji>')
    text = convert_fa_numbers(text)
    text = convert_ar_characters(text)
    # regex to detect and replace all smilies in the text with <smiley>
    text = re.sub(r"(:\s?\)|:-\)|\(\s?:|\(-:|:\'\)|:\s?D|8-\)|:\s?\||;\s?\)|:-\*|:-\||:-\(|:\s?P|:-P|:-p|:-b|:-O|:-o|:-0|:-\@|:\$|:-\^|:-&|:-\*|:-\+|:-\~|:-\`|:-\>|:-\<|:-\}|:-\{|\[:\s?\]|\[:\s?\]|:\s?\]|:\s?\[|:\s?\}|:\s?\{)",'<smiley>',text)
    text = text.lower() # we lowercase here to prevent changes in the URLs and smilies
    text = text.strip()
    text = re.sub(r'[<>#.:()"\'!?؟،,@$%^&*_+\[\]/]', ' ', text)
    text = re.sub(r'[\s]{2,}', ' ', text)
    text = re.sub(r'(\w)\1{2,}', r'\1',text)
    text = text.replace('\u200c', ' ')
    text = ' '.join([word for word in text.split() if word not in (stopwords)])
    if re.search(r'[\u0600-\u06FF]', text):
        return(text)
    else:
        return 'None'

In [ ]:
! git clone https://github.com/kharazi/persian-stopwords.git
%cd persian-stopwords/

Cloning into 'persian-stopwords'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 51 (delta 7), reused 5 (delta 5), pack-reused 42
Unpacking objects: 100% (51/51), 34.96 KiB | 994.00 KiB/s, done.
/content/persian-stopwords


In [ ]:
with open('persian', 'r') as f:
    stopwords = f.read().splitlines()

In [ ]:
tqdm.pandas()

In [ ]:
corpus['Cleaned'] = corpus['comment'].progress_apply(preprocess)

100%|██████████| 7000/7000 [03:20<00:00, 34.85it/s]


In [26]:
corpus.to_csv('snappfood_preprocessed.csv')

In [39]:
m_corpus = pd.read_csv("snappfood_preprocessed.csv")

In [40]:
m_corpus

,Unnamed: 0.1,Unnamed: 0,comment,label,label_id,Cleaned
0,1252,NaN,غذا سرد شده بود. حیف اون پیتزا که سرد شد,HAPPY,0.0,غذا سرد حیف پیتزا سرد
1,1949,NaN,مزه خورشت سبزی خوب نبود، غذا مزه‌ی سوختگی می‌د...,HAPPY,0.0,مزه خورشت سبزی غذا مزه سوختگی چلوجوجه
2,6977,NaN,غذاشون خوب بود ولی پول برنج ایرانی گرفتن برنج ...,SAD,1.0,غذاشون پول برنج ایرانی برنج هندی فرستادن
3,5275,NaN,بخش پیتزای پنجره‌ای خوش طعم اما بسیار نازک بود...,SAD,1.0,پیتزای پنجره طعم نازک کیفیت گوشت پلاس آروستو پ...
4,3944,NaN,کیک وانیلی سفارش داده بودم اصلا تازه و خوشمزه ...,SAD,1.0,کیک وانیلی سفارش خوشمزه
...,...,...,...,...,...,...
6995,1224,NaN,سلام و خسته نباشید مثل همیشه عالی، تازه و خوشم...,HAPPY,0.0,سلام خسته نباشید عالی خوشمزه توجه توضیحاتم تما...
6996,2598,NaN,بسیار زمان تحویل مناسب بود,HAPPY,0.0,تحویل مناسب
6997,923,NaN,مثل همیشه بی نظیر و عالی,HAPPY,0.0,عالی
6998,4025,NaN,سالاد سزار بسیار بی کیفیت بود …حیف اسمه به این...,SAD,1.0,سالاد سزار کیفیت …حیف اسمه خوشمزگی


In [41]:
corpus = m_corpus['Cleaned'] 

# **tf-idf**

In [42]:
def words_with_index(corpus):

  sentences = []
  word_set = []
  for sent in list(corpus):
      for word in word_tokenize(sent):
          if word not in word_set:
              word_set.append(word)
  #uniqe words
  word_set = set(word_set)
  #number of docs
  total_docs = len(list(corpus))

  word_index = {}
  for i, word in enumerate(word_set):
      word_index[word] = i

  return word_index, word_set , total_docs 


In [43]:
word_indx, word_set, total_docs = words_with_index(corpus)
print(word_indx)

{'میگوش': 0, 'رسانی': 1, 'مینوشتین': 2, 'محشره': 3, 'غذارو': 4, 'راهکار': 5, 'دایمتون': 6, 'باتعهد': 7, 'خیارهای': 8, 'سوخاریها': 9, 'فویل': 10, 'یوسف': 11, 'تیتابی': 12, 'حالمون': 13, 'ایزوله': 14, 'دوپر': 15, 'میمونه': 16, 'وسیب': 17, 'میشدن': 18, 'لذیذتر': 19, 'ابغاد': 20, 'گوجش': 21, 'محدودیت': 22, 'نفرم': 23, 'وان': 24, 'همینطوره': 25, 'نکردین': 26, 'پنیرشون': 27, 'دوس': 28, 'ثابتم': 29, 'بخار': 30, 'بیاد؛': 31, 'واین': 32, 'آپاچینو': 33, 'ضحم': 34, 'همکاراتون': 35, 'قطر': 36, 'برادر': 37, 'اشتباهی': 38, 'نعناداغ': 39, 'مقوای': 40, 'دایتی': 41, 'روگفتم': 42, 'بدند': 43, 'آمریکاییش': 44, 'نحو': 45, 'پیوستین': 46, 'برد': 47, 'وروغن': 48, 'خوشحالی': 49, '9500': 50, 'آوورده': 51, 'ندن': 52, 'تایی': 53, 'مریزاد': 54, 'میدونه': 55, 'وبسته': 56, 'کاهوهای': 57, 'موادغذایی': 58, 'بویه': 59, 'پشینهاد': 60, '18': 61, 'محیط': 62, 'نگیرم': 63, 'جردن': 64, 'حواسشون': 65, 'شاکی': 66, 'نظارت': 67, 'طلبکاره': 68, 'تصویری': 69, 'پور': 70, 'اگ': 71, 'یخ': 72, 'نمیفرسته': 73, 'مهمونش': 74, 'پیک': 75,

In [45]:
print(word_set)

{'میگوش', 'رسانی', 'مینوشتین', 'محشره', 'غذارو', 'راهکار', 'دایمتون', 'باتعهد', 'خیارهای', 'سوخاریها', 'فویل', 'یوسف', 'تیتابی', 'حالمون', 'ایزوله', 'دوپر', 'میمونه', 'وسیب', 'میشدن', 'لذیذتر', 'ابغاد', 'گوجش', 'محدودیت', 'نفرم', 'وان', 'همینطوره', 'نکردین', 'پنیرشون', 'دوس', 'ثابتم', 'بخار', 'بیاد؛', 'واین', 'آپاچینو', 'ضحم', 'همکاراتون', 'قطر', 'برادر', 'اشتباهی', 'نعناداغ', 'مقوای', 'دایتی', 'روگفتم', 'بدند', 'آمریکاییش', 'نحو', 'پیوستین', 'برد', 'وروغن', 'خوشحالی', '9500', 'آوورده', 'ندن', 'تایی', 'مریزاد', 'میدونه', 'وبسته', 'کاهوهای', 'موادغذایی', 'بویه', 'پشینهاد', '18', 'محیط', 'نگیرم', 'جردن', 'حواسشون', 'شاکی', 'نظارت', 'طلبکاره', 'تصویری', 'پور', 'اگ', 'یخ', 'نمیفرسته', 'مهمونش', 'پیک', 'دورچبن', 'گرونه', 'واقعیه', 'زدم', 'برگراتور', 'سردار', 'آیفون', 'ماکروفری', 'میزنه', 'صحبت', 'زنگ', 'بیخود', 'نمایید', 'نبودم', 'تهش', 'بصرفست', 'لابار', 'فهرست', 'ومیشن', 'متفاوت', 'اشتیاق', 'خریم', 'گفتار', 'میتونس', 'افراد', 'رانندگی', 'همینجوریش', 'ظاهر', 'برخلاف', 'انقضا', 'تیغ', 'برم'

In [47]:
def count_dict(sentences, word_set):
    count_dict = {}
    for word in word_set:
        count_dict[word] = 0
    for sent in sentences:
        for word in word_tokenize(sent):
            count_dict[word] += 1
    return count_dict
word_count = count_dict(list(corpus), word_set)
print(word_count)

{'میگوش': 1, 'رسانی': 4, 'مینوشتین': 2, 'محشره': 1, 'غذارو': 26, 'راهکار': 1, 'دایمتون': 3, 'باتعهد': 1, 'خیارهای': 1, 'سوخاریها': 1, 'فویل': 8, 'یوسف': 3, 'تیتابی': 2, 'حالمون': 1, 'ایزوله': 2, 'دوپر': 3, 'میمونه': 5, 'وسیب': 2, 'میشدن': 2, 'لذیذتر': 1, 'ابغاد': 1, 'گوجش': 1, 'محدودیت': 1, 'نفرم': 1, 'وان': 2, 'همینطوره': 2, 'نکردین': 3, 'پنیرشون': 1, 'دوس': 14, 'ثابتم': 1, 'بخار': 3, 'بیاد؛': 1, 'واین': 2, 'آپاچینو': 1, 'ضحم': 5, 'همکاراتون': 1, 'قطر': 1, 'برادر': 2, 'اشتباهی': 10, 'نعناداغ': 1, 'مقوای': 2, 'دایتی': 1, 'روگفتم': 1, 'بدند': 3, 'آمریکاییش': 1, 'نحو': 1, 'پیوستین': 1, 'برد': 10, 'وروغن': 1, 'خوشحالی': 1, '9500': 1, 'آوورده': 1, 'ندن': 2, 'تایی': 4, 'مریزاد': 2, 'میدونه': 2, 'وبسته': 1, 'کاهوهای': 4, 'موادغذایی': 1, 'بویه': 1, 'پشینهاد': 1, '18': 2, 'محیط': 5, 'نگیرم': 1, 'جردن': 1, 'حواسشون': 1, 'شاکی': 2, 'نظارت': 4, 'طلبکاره': 1, 'تصویری': 1, 'پور': 1, 'اگ': 3, 'یخ': 89, 'نمیفرسته': 1, 'مهمونش': 1, 'پیک': 469, 'دورچبن': 1, 'گرونه': 11, 'واقعیه': 1, 'زدم': 12, 'برگراتو

In [48]:
def t_f(document, word):
    N = len(document)
    occurance = len([token for token in document if token == word])
    return occurance / N

def i_d_f(word):
  if word_count[word] == 0:
    word_occurance = 1
  else:
    word_occurance = word_count[word] + 1

  return np.log(total_docs / word_occurance)

def tf_idf(sentence):
    vec = np.zeros((len(word_set),))
    for word in word_tokenize(sentence):
        tf = t_f(word_tokenize(sentence), word)
        idf = i_d_f(word)
        vec[word_indx[word]] = tf * idf
    return vec

In [49]:
vectors = []
for sent in list(corpus):
    vectors.append(tf_idf(sent))


In [50]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vectors, m_corpus['label_id'], test_size=0.1,random_state = 42)

In [51]:
from sklearn.naive_bayes import MultinomialNB
NB = MultinomialNB()
model = NB.fit(X_train,y_train)

In [52]:
y_pred = model.predict(X_test)

In [53]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.83      0.77      0.79       354
         1.0       0.78      0.84      0.81       346

    accuracy                           0.80       700
   macro avg       0.80      0.80      0.80       700
weighted avg       0.80      0.80      0.80       700



# **ppmi**

In [54]:
def count_word_in_sentence(word,sentence):
  
  c = 0
  for w in word_tokenize(sentence):
    if w == word:
      c+=1
  return c

In [55]:
corpus_size = len(corpus)
sentences = list(corpus)

def ppmi(sentence, word_set, word_index, word_count):
  ppmi_list = len(word_set)*[0]
  sentence_size= len(sentence)

  for w in word_tokenize(sentence):
    

    p_w = word_count[w]/corpus_size
    p_c = sentence_size/corpus_size
    p_c_w = count_word_in_sentence(w,sentence)/corpus_size
    w_pmi = (p_w*p_c)/p_c_w
    if w_pmi < 0:
      w_pmi = 0
    
    ppmi_list[word_index[w]] = w_pmi
    
  return ppmi_list



In [56]:
vectors_ppmi = []
for sent in list(corpus):
  vectors_ppmi.append(ppmi(sent, word_set, word_indx, word_count))
  


In [57]:
len(vectors_ppmi)

7000

In [58]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vectors_ppmi, m_corpus['label_id'], test_size=0.1,random_state = 42)

In [59]:
from sklearn.naive_bayes import MultinomialNB
NB = MultinomialNB()
model = NB.fit(X_train,y_train)

In [60]:
y_pred = model.predict(X_test)

In [61]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.89      0.62      0.73       354
         1.0       0.71      0.92      0.80       346

    accuracy                           0.77       700
   macro avg       0.80      0.77      0.77       700
weighted avg       0.80      0.77      0.77       700

